### <strong>主題:
啤酒評論評分預測 - 分詞器(Tokenizer)
### <strong>說明:
繼續上次啤酒的評鑑資料集的練習，還記得在上次的資料初步分析之後，我們對於BERT模型最大 <br />
長度的限制是255嗎? 然而這還只是未使用分詞器所得到的結論，當真正使用分詞器之後，每個 <br />
評論具的token數可能會與我們先前的評估有所不同，因此這是作業主要是以BERT分瓷器來驗證 <br />
上次作業得到的結論是否需要修正。
### <strong>題目
1. 創建英文BERT所使用的分詞器，提供下一題分析以及後續訓練使用
2. 以分詞器生成每個評論語句的token，並評估上次得到的最大長度限制是否合理

#### <strong>提示: 不要忘記加上[SEP]與[CLS]

In [1]:
# 連接個人資料 讀取 ＰＴＴ 訓練資料和儲存模型
#先連接自己的GOOGLE DRIVE 為了要儲存資料和訓練模型
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days-part2/project_2_2/')
print(os.getcwd())

/content
/content/drive/MyDrive/python_training/NLP100Days-part2/project_2_2


In [3]:
!pip install torch
!pip install transformers
#!pip install -q transformers
# 設定 torchtext 版本 安裝完必須重新啟動執行階段
!pip install torchtext==0.6.0

     |████████████████████████████████| 2.6 MB 6.0 MB/s 
     |████████████████████████████████| 636 kB 68.0 MB/s 
     |████████████████████████████████| 3.3 MB 43.4 MB/s 
     |████████████████████████████████| 895 kB 74.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 64 kB 1.7 MB/s 
     |████████████████████████████████| 1.2 MB 9.1 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [4]:
import torch
import transformers
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from transformers import BertModel, BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup

In [5]:
TRAIN = pd.read_json("./data/train_set_lng.json")
TEST = pd.read_json("./data/test_set_lng.json")

In [6]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
TOKENIZER = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [7]:
text = TRAIN['review/text'][0][:250]

tokens = TOKENIZER.tokenize(text)
token_ids = TOKENIZER.convert_tokens_to_ids(tokens)

print(text)
print(tokens[:50])
print(token_ids[:50])

Poured from 12oz bottle into chalice.	No B.O.D. on bottle		Aroma: Bright citrusy hops, biscuity malt, a faint green apple twang, followed by pineapple and grapefruit on the end.		Appearance: Beautiful clear (for style) yellow-golden body. 2-finger he
['Po', '##ured', 'from', '12', '##oz', 'bottle', 'into', 'ch', '##ali', '##ce', '.', 'No', 'B', '.', 'O', '.', 'D', '.', 'on', 'bottle', 'A', '##roma', ':', 'Bright', 'c', '##itrus', '##y', 'hop', '##s', ',', 'bi', '##s', '##cu', '##ity', 'ma', '##lt', ',', 'a', 'faint', 'green', 'apple', 't', '##wang', ',', 'followed', 'by', 'pine', '##ap', '##ple', 'and']
[18959, 10105, 1121, 1367, 17395, 5346, 1154, 22572, 10584, 2093, 119, 1302, 139, 119, 152, 119, 141, 119, 1113, 5346, 138, 18885, 131, 13603, 172, 22220, 1183, 6974, 1116, 117, 16516, 1116, 10182, 1785, 12477, 6066, 117, 170, 7859, 2448, 12075, 189, 15946, 117, 1723, 1118, 10194, 11478, 7136, 1105]


In [8]:
print(TOKENIZER.sep_token, TOKENIZER.sep_token_id)
print(TOKENIZER.cls_token, TOKENIZER.cls_token_id)
print(TOKENIZER.pad_token, TOKENIZER.pad_token_id)
print(TOKENIZER.unk_token, TOKENIZER.unk_token_id)

[SEP] 102
[CLS] 101
[PAD] 0
[UNK] 100


In [9]:
MAX_SEQ_LEN = 512

In [10]:

def get_encoding(text, max_seq_len=MAX_SEQ_LEN):
    encoding = TOKENIZER.encode_plus(
        text,
        max_length=max_seq_len,
        truncation=True,
        add_special_tokens=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    return [t for t in encoding['input_ids'][0] if t != 0]##encoding["attention_mask"]

In [11]:
TRAIN['token_number'] = TRAIN['review/text'].apply(str).apply(get_encoding).apply(len)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [12]:
TRAIN['token_number'].describe([0.8, 0.9, 0.95])

count    45000.000000
mean       182.230667
std         90.194083
min          3.000000
50%        161.000000
80%        242.000000
90%        303.000000
95%        365.000000
max        512.000000
Name: token_number, dtype: float64

#結論

Tokenize 後可以發現語句的 Token 數超出先前使用空白分割的 256 (因為 BERT 使用 word piece)
